In [1]:
import pathlib
import os

from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds
import tensorflow as tf

from misc.CycleGANTraining import CycleGANTraining
from misc.train_monitor import get_train_monitor
from misc.data import read_train_sample
from misc.data import read_test_sample
from misc.model_tf import CycleGAN
from misc import configs

AUTO = tf.data.AUTOTUNE
tf.random.set_seed(42)

2023-06-07 07:06:22.450972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 07:06:23.168818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/envs/dove/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## 데이터 셋 다운로드 받는 부분
datasets = tfds.load('cycle_gan/apple2orange')
(train_input, train_output) = datasets['trainA'], datasets['trainB']
(test_input ,  test_output) = datasets['testA'] , datasets['testB'] 

def dataset_map(dataset, func, batch_size, dtype = 'train'): 
    
    dataset = dataset.map(func, num_parallel_calls = AUTO)
    dataset = dataset.shuffle(batch_size).batch(batch_size).repeat() if dtype == 'train' else dataset.shuffle(batch_size).batch(batch_size)
    return dataset

## 데이터 셋 전처리
train_input  = dataset_map(train_input , read_train_sample, configs.TRAIN_BATCH_SIZE)
train_output = dataset_map(train_output, read_train_sample, configs.TRAIN_BATCH_SIZE)

test_input   = dataset_map(test_input  , read_test_sample, configs.INFER_BATCH_SIZE, dtype = 'test')
test_output  = dataset_map(test_output , read_test_sample, configs.INFER_BATCH_SIZE, dtype = 'test')

train_dataset = tf.data.Dataset.zip((train_input, train_output))
loss          = tf.keras.losses.BinaryCrossentropy(from_logits = True)
model         = CycleGAN(configs.IMG_HEIGHT, configs.IMG_WIDTH)

2023-06-07 07:06:29.578358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 34441 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-06-07 07:06:29.581322: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38370 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2023-06-07 07:06:29.587507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38370 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capability: 8.0
2023-06-07 07:06:29.590367: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 38370 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-40GB, pci bu

In [3]:
discX = model.discriminator()
discY = model.discriminator()
genG  = model.generator()
genF  = model.generator()

os.makedirs(configs.BASE_IMAGES_PATH, exist_ok = True)

cycleGAN = CycleGANTraining(
                    gen_G = genG, disc_X = discX,
                    gen_F = genF, disc_Y = discY
            )

cycleGAN.compile(
                    g_optimG = Adam(learning_rate = configs.LR),
                    d_optimX = Adam(learning_rate = configs.LR),
                    g_optimF = Adam(learning_rate = configs.LR),
                    d_optimY = Adam(learning_rate = configs.LR),
                    bceLoss  = loss
                )

callbacks = [get_train_monitor(test_input, test_output, epoch_interval = 10,
                               image_path = configs.BASE_IMAGES_PATH,
                               batch_size = configs.INFER_BATCH_SIZE)]

(None, 256, 256, 3)
(None, 128, 128, 64)
(None, 64, 64, 128)
(None, 32, 32, 256)
(None, 32, 32, 512)
(None, 30, 30, 1)


------------


(None, 256, 256, 3)
(None, 128, 128, 64)
(None, 64, 64, 128)
(None, 32, 32, 256)
(None, 32, 32, 512)
(None, 30, 30, 1)


------------




2023-06-07 07:06:30.630293: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-06-07 07:06:30.630782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-06-07 07:06:30.744070: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline 

In [4]:
cycleGAN.fit(train_dataset, epochs = configs.EPOCHS, callbacks = callbacks,
             steps_per_epoch = configs.STEPS_PER_EPOCH)

cycleGAN.gen_G.save(config.GENERATOR_MODEL)

Epoch 1/50


2023-06-07 07:06:30.899353: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-06-07 07:06:30.900083: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-06-07 07:06:36.949421: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-06-07 07:06:38.839432: E tensorflow/compiler/xla/st

UnimplementedError: Graph execution error:

Detected at node 'model_1/conv2d_5/Conv2D' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/opt/conda/envs/dove/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/envs/dove/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/envs/dove/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/opt/conda/envs/dove/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/envs/dove/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/opt/conda/envs/dove/lib/python3.11/asyncio/base_events.py", line 1922, in _run_on